https://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.Y2cy3nZBzIX

https://github.com/kavgan/nlp-in-practice/blob/master/word2vec/Word2Vec.ipynb

https://github.com/kavgan/nlp-in-practice/blob/master/word2vec/Word2Vec.ipynb

https://github.com/kavgan/nlp-in-practice/blob/master/word2vec/Word2Vec.ipynb

In [ ]:
# imports needed and logging
import gzip
import gensim 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [ ]:
from google.colab import drive
drive.mount("/Similar_Word")

Mounted at /Similar_Word


In [ ]:

#!unzip /Similar_Word/MyDrive/NLP/Distance_Between_two_words/OpinRankDatasetWithJudgments.zip -d /Similar_Word/MyDrive/NLP/Distance_Between_two_words
            

In [ ]:

input_file = "/Similar_Word/MyDrive/NLP/Distance_Between_two_words/reviews_data.txt.gz"
with gzip.open (input_file, 'rb') as f:
        for i,line in enumerate (f):
            print(line)
            break



b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

In [ ]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (input_file))
logging.info ("Done reading data file")


In [ ]:

model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)


(303504528, 415193580)

In [ ]:

w1 = "dirty"
model.wv.most_similar (positive=w1)

[('filthy', 0.8618006110191345),
 ('stained', 0.7831227779388428),
 ('unclean', 0.773486852645874),
 ('dusty', 0.7694820761680603),
 ('smelly', 0.7617112398147583),
 ('grubby', 0.7551862597465515),
 ('grimy', 0.7347296476364136),
 ('dingy', 0.7327849864959717),
 ('soiled', 0.7295088171958923),
 ('mouldy', 0.7270488739013672)]

In [ ]:

# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9261513948440552),
 ('friendly', 0.8334428668022156),
 ('cordial', 0.811875581741333),
 ('professional', 0.7865085005760193),
 ('attentive', 0.770498514175415),
 ('curteous', 0.7600521445274353)]

In [ ]:

# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.8125640749931335),
 ('amazed', 0.7998853921890259),
 ('appalled', 0.7774959802627563),
 ('astonished', 0.7673192024230957),
 ('stunned', 0.7485215067863464),
 ('dismayed', 0.737923264503479)]

In [ ]:

# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('duvet', 0.7087066173553467),
 ('blanket', 0.6890629529953003),
 ('mattress', 0.6853978633880615),
 ('matress', 0.667702317237854),
 ('quilt', 0.6663302779197693),
 ('pillowcase', 0.6542825698852539),
 ('sheets', 0.6388118267059326),
 ('foam', 0.635508120059967),
 ('pillows', 0.6337054371833801),
 ('pillowcases', 0.6203763484954834)]

**Similarity between two words in the vocabulary**

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary.

In [ ]:

# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")


0.7617112

In [ ]:

# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")


1.0

In [ ]:

# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")


0.27082956

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that dirty is highly similar to smelly but dirty is dissimilar to clean. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring here.

**Find the odd one out**

You can even use Word2Vec to find odd items given a list of items.

In [ ]:

# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'shower'

**Understanding some of the parameters**

To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.


model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)

**size**

The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me.

**window**

The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window.

**min_count**

Minimium frequency count of words. The model would ignore words that do not statisfy the min_count. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

**workers**

How many threads to use behind the scenes?

**When should you use Word2Vec?**

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary.


Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work.

In [ ]:
# https://www.educative.io/answers/how-to-find-similarity-between-two-words-using-nlp
# https://python.gotrained.com/nltk-edit-distance-jaccard-distance/
# https://www.kdnuggets.com/2018/08/emotion-sentiment-analysis-practitioners-guide-nlp-5.html
# https://www.kdnuggets.com/2022/10/10-cheat-sheets-need-ace-data-science-interview.html
# https://monkeylearn.com/sentiment-analysis/
# https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55
# https://colab.research.google.com/github/shankygupta79/GoogleCollabFiles/blob/master/Word2Vec.ipynb#scrollTo=HDjdWIER5Whn
# https://soumilshah1995.blogspot.com/2020/05/lets-build-simple-nlp-model-that.html


https://www.educative.io/answers/how-to-find-similarity-between-two-words-using-nlp

https://www.educative.io/answers/how-to-find-similarity-between-two-words-using-nlp

https://www.educative.io/answers/how-to-find-similarity-between-two-words-using-nlp

In [ ]:

import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

vec_king = wv['king']

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:

!pip install gensim
!pip install scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

!pip install gensim
!pip install scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
print('Imported Successfully!')

Imported Successfully!


In [ ]:

v_apple = wv['banana']
v_mango = wv['mango']
cosine_similarity([v_apple],[v_mango])


array([[0.63652116]], dtype=float32)

In [ ]:

v_apple = wv['serve']
v_mango = wv['service']
cosine_similarity([v_apple],[v_mango])


array([[0.32593498]], dtype=float32)

In [ ]:

v_apple = wv['happy']
v_mango = wv['unhappy']
cosine_similarity([v_apple],[v_mango])


array([[0.6128039]], dtype=float32)